In [11]:
import time
import jwt
import requests
import json
import pdfplumber
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 对话API
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )
    
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("2fa1e0dfc28b5946702e62a5fd058e29.LEe7azoO7oKolOKT", 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": (content)}]
    }
    
    response = requests.post(url, headers=headers, json=data)
    print("【大模型返回】")
    print(response)
    return response.json()

bge = json.load(open("submit_bge_t10.json",encoding='utf-8'))
bm25 = json.load(open("submit_bm25_t10.json",encoding='utf-8'))

#读取问答数据集
questions = json.load(open("questions.json",encoding='utf-8'))
pdf = pdfplumber.open("初赛训练数据集.pdf")
pdf_content_dict = {}
#len(pdf.pages) # 页数
for page_idx in range(len(pdf.pages)):
    pdf_content_dict['page_' + str(page_idx + 1)] = pdf.pages[page_idx].extract_text()

#做两路召回的重排序
fusion_result = []
k=60
#进行排序的方式打分
for q1,q2 in zip(bge,bm25):
    fusion_score = {}
    for idx,q in enumerate(q1['reference']):
        if q not in fusion_score:
            fusion_score[q] = 1/ (idx + k)
        else:
            fusion_score[q] += 1 / (idx + k)

    for idx,q in enumerate(q2['reference']):
        if q not in fusion_score:
            fusion_score[q] = 1/ (idx + k)
        else:
            fusion_score[q] += 1 / (idx + k)
    #得到打分排序之后的结果
    sorted_dict = sorted(fusion_score.items(), key=lambda item:item[1], reverse=True)
    q1['reference'] = sorted_dict[0][0]

    reference_pages = [int(x[0].split('_')[1]) for x in sorted_dict]
    reference_pages = np.array(reference_pages)

    reference_content = ''
    reference_content += pdf_content_dict[sorted_dict[0][0]].replace('\n',' ')+"上述内容在第"+str(reference_pages)+"页"

    print("【用户提问】\n" + q1["question"])
    print("【参考资料】\n" + pdf_content_dict[q1['reference']])

    reference_page = q1['reference'].split('_')[1]
    #构造提示词
    prompt = '''你是一个汽车专家，帮我结合给定的资料，回答一个问题。如果问题无法从资料中获得，请输出结合给定的资料，无法回答问题。
资料：{0}

问题：{1}
    '''.format(reference_content, q1["question"])
    answer = "无法"
    #避免单次调用失败，调用五次
    for _ in range(5):
        try:
            answer = ask_glm(prompt)['choices'][0]['message']['content']
            if answer:
                break
        except:
                print('111')
                continue
            
    if '无法' in answer:
        answer = '结合给定资料，无法回答问题'
    
    q1['answer'] = answer
    print("【模型回答】\n" + q1['answer'])
    print("\n\n\n")

    fusion_result.append(q1)

with open('submit_bge+bm25_rerank_retrieval_glm4.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)




【用户提问】
“前排座椅通风”的相关内容在第几页？
【参考资料】
安全出行
在中央显示屏中点击 ，进入驾驶员侧座椅通风控制界面。
01设置副驾驶员侧座椅通风强度及开关控制。
驾驶员/副驾驶员侧座椅通风分三级调节，点击控制开关后在“关-低-
01设置驾驶员侧座椅通风强度及开关控制。
中-高”之间循环。
在中央显示屏中点击 ，进入副驾驶员侧座椅通风控制界面。
使用Lynk&CoApp打开/关闭前排座椅通风
打开/关闭前排座椅通风图标：登录Lynk&CoApp，按下
该图标可以打开/关闭前排座椅通风。
设置前排座椅通风时间
在中央显示屏中唤起空调控制界面，然后点击 -舒适。
116
【大模型返回】
<Response [200]>
【模型回答】
根据提供的资料，前排座椅通风的相关内容在第116页。




【用户提问】
"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"
【参考资料】
安全出行
警告！
■ 行程中儿童的安全需要由同乘大人负责。
■ 切勿让儿童站在或跪坐在座椅上，也不得让儿童待在后备厢区
域。
■ 切勿将儿童单独留在车内无人照看，切勿让儿童携带钥匙。
■ 行车前，请启用儿童锁，以防儿童打开车门或意外操作电动车
窗。
■ 请将儿童放置在安全座椅上，不允许将儿童抱在怀中。
■ 禁止将儿童安全座椅安装到前排乘客座椅上。
■ 身高低于140cm的乘客不得坐在前排乘客座椅上。
ISOFIX固定装置
安装ISOFIX儿童安全座椅 1 打开固定装置罩盖。
安装儿童安全座椅或婴儿座椅时，请务必遵守当地法规，并查阅制造 2按照儿童安全座椅安装说明书，将儿童安全座椅安装在ISOFIX固
商说明书。请将儿童安全座椅安装说明书放入手套箱内，以便随时查 定装置中。
阅。
3向外拉动并左右摇晃儿童安全座椅，以确保其已固定到位。
使用顶部系带固定装置
车辆配备有儿童安全座椅顶部系带固定装置，位于靠背的后侧。
请按照儿童安全座椅安装说明书，固定儿童安全座椅。
123
【大模型返回】
<Response [200]>
【模型回答】
根据您提供的资料，关于车辆的儿童安全座椅固定装置的相关内容可以在第[351 107 4 11 125 273 80 271 102 226 101 339 29 112]页找到。




【用户提问】
“打开前机舱盖”的相关信息在第几页？
【参考资料】